In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics, losses

2023-09-20 18:21:47.956314: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 1.数据集准备
(x, y), (x_val, y_val) = datasets.mnist.load_data()  # 加载数据集，返回的是两个元组，分别表示训练集和测试集
x = tf.convert_to_tensor(x, dtype=tf.float32)/255.  # 转换为张量，并缩放到0~1
y = tf.convert_to_tensor(y, dtype=tf.int32)  # 转换为张量（标签）
print(x.shape, y.shape)
train_dataset = tf.data.Dataset.from_tensor_slices((x, y))  # 构建数据集对象
train_dataset = train_dataset.batch(32).repeat(10)  # 设置批量训练的batch为32，要将训练集重复训练10遍

(60000, 28, 28) (60000,)


In [3]:
# 2.搭建网络
network = Sequential([  # 搭建网络容器
    layers.Conv2D(6, kernel_size=3, strides=1),  # 第一个卷积层，6个3*3*1卷积核
    layers.MaxPooling2D(pool_size=2, strides=2),  # 池化层，卷积核2*2，步长2
    layers.ReLU(),  # 激活函数
    layers.Conv2D(16, kernel_size=3, strides=1),  # 第二个卷积层，16个3*3*6卷积核
    layers.MaxPooling2D(pool_size=2, strides=2),  # 池化层
    layers.ReLU(),  # 激活函数
    layers.Flatten(),  # 拉直，方便全连接层处理

    layers.Dense(120, activation='relu'),  # 全连接层，120个节点
    layers.Dense(84, activation='relu'),  # 全连接层，84个节点
    layers.Dense(10)  # 输出层，10个节点
])
network.build(input_shape=(None, 28, 28, 1))  # 定义输入,batch_size=32,输入图片大小是28*28,通道数为1。
network.summary()  # 显示出每层的待优化参数量

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 6)         60        
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 6)         0         
 D)                                                              
                                                                 
 re_lu (ReLU)                (None, 13, 13, 6)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 16)        880       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 16)          0         
 g2D)                                                            
                                                                 
 re_lu_1 (ReLU)              (None, 5, 5, 16)          0

In [5]:
# 3.模型训练（计算梯度，迭代更新网络参数）
optimizer = optimizers.SGD(lr=0.01)  # 声明采用批量随机梯度下降方法，学习率=0.01
acc_meter = metrics.Accuracy()  # 新建accuracy测量器
for step, (x, y) in enumerate(train_dataset):  # 一次输入batch组数据进行训练
    with tf.GradientTape() as tape:  # 构建梯度记录环境
        x = tf.reshape(x, (32, 28, 28, 1))  # 将输入拉直，[b,28,28]->[b,784]
        # x = tf.extand_dims(x, axis=3)
        out = network(x)  # 输出[b, 10]
        y_onehot = tf.one_hot(y, depth=10)  # one-hot编码
        loss = tf.square(out - y_onehot)
        loss = tf.reduce_sum(loss)/32  # 定义均方差损失函数，注意此处的32对应为batch的大小
        grads = tape.gradient(loss, network.trainable_variables)  # 计算网络中各个参数的梯度
        optimizer.apply_gradients(zip(grads, network.trainable_variables))  # 更新网络参数
        acc_meter.update_state(tf.argmax(out, axis=1), y)  # 比较预测值与标签，并计算精确度（写入数据，进行求精度）

    if step % 200 == 0:  # 每200个step，打印一次结果
        print('Step', step, ': Loss is: ', float(loss), ' Accuracy: ', acc_meter.result().numpy())  # 读取数据
        acc_meter.reset_states()  # 清零测量器l


Step 0 : Loss is:  1.0376428365707397  Accuracy:  0.15625
Step 200 : Loss is:  0.6368505954742432  Accuracy:  0.4475
Step 400 : Loss is:  0.49935227632522583  Accuracy:  0.75140625
Step 600 : Loss is:  0.3866470754146576  Accuracy:  0.8371875
Step 800 : Loss is:  0.3106642961502075  Accuracy:  0.8778125
Step 1000 : Loss is:  0.40175342559814453  Accuracy:  0.8853125
Step 1200 : Loss is:  0.29926443099975586  Accuracy:  0.9071875
Step 1400 : Loss is:  0.22922320663928986  Accuracy:  0.91375
Step 1600 : Loss is:  0.2142447829246521  Accuracy:  0.91765624
Step 1800 : Loss is:  0.242709681391716  Accuracy:  0.93578124
Step 2000 : Loss is:  0.23537187278270721  Accuracy:  0.9501563
Step 2200 : Loss is:  0.1548382192850113  Accuracy:  0.93421876
Step 2400 : Loss is:  0.22432342171669006  Accuracy:  0.9348438
Step 2600 : Loss is:  0.16739065945148468  Accuracy:  0.94625
Step 2800 : Loss is:  0.15792438387870789  Accuracy:  0.94171876
Step 3000 : Loss is:  0.23086220026016235  Accuracy:  0.942

In [10]:
#4
# 模型配置
network.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
# 打印网络参数
print(network.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 6)         60        
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 6)         0         
 D)                                                              
                                                                 
 re_lu (ReLU)                (None, 13, 13, 6)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 16)        880       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 16)          0         
 g2D)                                                            
                                                                 
 re_lu_1 (ReLU)              (None, 5, 5, 16)          0

In [11]:
# 训练
history = network.fit(x, y, batch_size=64, epochs=10, validation_split=0.3)

Epoch 1/10
657/657 [==============================] - 13s 18ms/step - loss: 2.3026 - accuracy: 0.1545 - val_loss: 2.3026 - val_accuracy: 0.1586
Epoch 2/10
657/657 [==============================] - 11s 17ms/step - loss: 2.3026 - accuracy: 0.1545 - val_loss: 2.3026 - val_accuracy: 0.1586
Epoch 3/10
657/657 [==============================] - 11s 17ms/step - loss: 2.3026 - accuracy: 0.1545 - val_loss: 2.3026 - val_accuracy: 0.1586
Epoch 4/10
657/657 [==============================] - 11s 17ms/step - loss: 2.3026 - accuracy: 0.1545 - val_loss: 2.3026 - val_accuracy: 0.1586
Epoch 5/10
657/657 [==============================] - 12s 19ms/step - loss: 2.3026 - accuracy: 0.1545 - val_loss: 2.3026 - val_accuracy: 0.1586
Epoch 6/10
657/657 [==============================] - 15s 23ms/step - loss: 2.3026 - accuracy: 0.1545 - val_loss: 2.3026 - val_accuracy: 0.1586
Epoch 7/10
657/657 [==============================] - 13s 20ms/step - loss: 2.3026 - accuracy: 0.1545 - val_loss: 2.3026 - val_accuracy:

In [12]:
#4
# 模型配置
network.compile(optimizer=tf.keras.optimizers.SGD(0.01), loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
# 打印网络参数
print(network.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 6)         60        
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 6)         0         
 D)                                                              
                                                                 
 re_lu (ReLU)                (None, 13, 13, 6)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 16)        880       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 16)          0         
 g2D)                                                            
                                                                 
 re_lu_1 (ReLU)              (None, 5, 5, 16)          0

In [13]:
history = network.fit(train_dataset, batch_size=32, epochs=5)

Epoch 1/5
18750/18750 [==============================] - 236s 13ms/step - loss: 2.3022 - accuracy: 0.1558
Epoch 2/5
18750/18750 [==============================] - 309s 16ms/step - loss: 2.3022 - accuracy: 0.1558
Epoch 3/5
18750/18750 [==============================] - 1339s 71ms/step - loss: 2.3022 - accuracy: 0.1558
Epoch 4/5
18750/18750 [==============================] - 2438s 130ms/step - loss: 2.3022 - accuracy: 0.1558
Epoch 5/5
18750/18750 [==============================] - 1843s 98ms/step - loss: 2.3022 - accuracy: 0.1558
